In [1]:
import os

import numpy as np
import pandas as pd

import asyncio

In [2]:
from datetime import datetime, timedelta

# objective to simulate for
objective = "RETIREMENT"

# define date range for simulation
freq = timedelta(days=30) # one month
n = 36 # three years
end = datetime.now()
date_range = pd.date_range(start=end-n*freq, end=end, freq=freq, normalize=True)

In [36]:
from dotenv import load_dotenv
from data import StockDataClient

load_dotenv()

client = StockDataClient()

if os.path.exists('testing.csv'):
    df = pd.read_csv('testing.csv')
    # add 'expReturn' column
    df["expReturn"] = df["priceTarget"] / df["price"] - 1
else:
    # Step 1. Get all stocks 
    stock_list = await client.get_all_stocks_by_exchange("NASDAQ")
    # Step 2. Filter stocks by market cap and get company profile
    min_cap = 50_000_000_000
    tasks = []
    for stock in stock_list:
        if stock.get("marketCap"):
            if stock["marketCap"] > min_cap:
                tasks.append(asyncio.create_task(client.get_company_profile(stock["symbol"])))

    filtered_stocks = await asyncio.gather(*tasks)

In [4]:
from params import OBJECTIVE_MAP

if not os.path.exists('testing.csv'):
    # Step 3 select a random subset of 50 stocks subject to sector constraints
    sector_allocations = OBJECTIVE_MAP[objective]["sector_allocations"]
    if not os.path.exists('testing.csv'):
        size = 50
        universe = pd.DataFrame.from_records(filtered_stocks)
        df = pd.DataFrame(columns=universe.columns)
        for sector, group in universe.groupby("sector"):
            if not sector in sector_allocations:
                continue
            # remove etfs and funds
            subset = group[(group["isEtf"] == False) & (group["isFund"] == False)]
            num = int(min((np.ceil(sector_allocations.get(sector) * size), len(subset))))
            df = pd.concat([df, subset.sample(num)])

In [5]:
if not os.path.exists('testing.csv'):
    df = pd.DataFrame(columns=["beta", "price", "priceTarget"], index=pd.MultiIndex.from_product([pd.Series(date_range, name="date"), df["symbol"]]))
    for _, stock in df.iterrows():
        tasks = [
            asyncio.create_task(client.get_analyst_research(stock["symbol"])),
            asyncio.create_task(client.get_historical_price(stock["symbol"])),
        ]
        research, price = await asyncio.gather(*tasks)
        
        for date in date_range:
            row = {
                "beta": stock["beta"], # no way to get historical beta, just use current
                "sector": stock["sector"],
                "price": None,
                "priceTarget": None,
            }
            # find price at date
            for i in range(1, len(price["historical"])):
                item = price["historical"][-i]
                if datetime.strptime(item["date"], "%Y-%m-%d") >= date:
                    row["price"] = item["close"]
                    break
            # find price target at date
            for i in range(1, len(research)):
                article = research[-i]
                publishedDate = datetime.strptime(article["publishedDate"].split("T")[0], "%Y-%m-%d")
                if publishedDate >= date:
                    break
                row["priceTarget"] = article["priceTarget"]
                
            df.loc[(date, stock["symbol"]),:] = row
            
    # save csv to local file
    df.to_csv('testing.csv')
    
    # add 'expReturn' column
    df["expReturn"] = df["priceTarget"] / df["price"] - 1        

# print df
print(df)

      Unnamed: 0 symbol   beta   price  priceTarget
0     2021-07-16    LIN  0.962  290.07        330.0
1     2021-07-16   META  1.206  341.16          NaN
2     2021-07-16   GOOG  1.010  131.85          NaN
3     2021-07-16   TMUS  0.497  149.41        139.0
4     2021-07-16   NFLX  1.257  530.31        617.0
...          ...    ...    ...     ...          ...
1734  2024-06-30   CRWD  1.074     NaN        410.0
1735  2024-06-30   LRCX  1.466     NaN        980.0
1736  2024-06-30   COIN  3.441     NaN        315.0
1737  2024-06-30   CSCO  0.845     NaN         50.0
1738  2024-06-30    CEG  0.794     NaN        218.0

[1739 rows x 5 columns]


In [41]:
# get benchmark
benchmark_data = await client.get_historical_price("^XNDX", date_range[0].strftime("%Y-%m-%d"), date_range[-1].strftime("%Y-%m-%d"))

In [50]:
from scipy.optimize import minimize, Bounds, LinearConstraint

def inv_utility_function(
    a: np.ndarray|pd.Series,
    df: pd.DataFrame,
):
    """
    Inverse utility function for optimisation.
    """
    r_f = 0.05 # TO DO
    # Treynor ratio = (r_p - r_f) / Beta_p, see https://www.investopedia.com/terms/t/treynorratio.asp
    r_p = np.dot(a, df["expReturn"].fillna(0))
    Beta_p = np.dot(a, df["beta"].fillna(1))
    # prevent divide by zero by adding small amount to beta
    U = (r_p - r_f) / (Beta_p if Beta_p != 0 else 0.01)
    return -U

initial_value = 1000 # set an initial value of 1000
benchmark = initial_value
portfolio = pd.DataFrame({"symbol": df["symbol"].unique(), "units": np.zeros(len(df["symbol"].unique()))}) # rolling copy of portfolio
iter = 0
for date, data in df.groupby('date'):
    # calculate portfolio value
    merged = pd.merge(data, portfolio[["symbol", "units"]], on='symbol', how='left')
    # fill na
    merged["units"] = merged["units"].fillna(0)
    value = np.sum(portfolio["units"] * merged["price"]) if iter > 0 else initial_value
    print(value, benchmark)
    
    # define bounds
    lb = np.zeros(len(merged))
    # upper bound equal to total value of portfolio
    ub = value * np.ones(len(merged))
    # set keep_feasible True to ensure iterations remain within bounds
    bnds = Bounds(lb, ub, keep_feasible=True)
    
    # define constraints
    # sum constraint
    sum_cons = [LinearConstraint(np.ones(len(merged)), value, value)]
    # sector constraints
    sector_cons = []
    # avoid multi collinearity by removing one of the sectors
    # target_sectors = list(OBJECTIVE_MAP[objective]["sector_allocations"].items())[:-1]

    # # seem to have better results by defining constraint for each sector individually
    # for sector, target in target_sectors:
    #     # array indicating which stocks belong to each sector
    #     sector_vector = np.array(df["sector"] == sector).astype(int)
    #     sector_cons.append(LinearConstraint(sector_vector, value*max(0, target-0.025), value*min(1, target+0.025)))

    
    initial_guess = value * np.ones(len(merged)) / len(merged)
    a = minimize(inv_utility_function, initial_guess, args=(merged),
                method='SLSQP', bounds=bnds, constraints=tuple(sum_cons+sector_cons),
                options={'maxiter': 100}).x
    
    portfolio["units"] = a / merged["price"]
        
    iter += 1

1000 1000


KeyboardInterrupt: 